In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from utils.SN_Curve import SN_Curve_qats
from utils.create_geo_matrix import create_geo_matrix
from utils.transformations import global_2_compass, compass_2_global
import numpy as np
import sys
import pandas as pd
import os
from utils.get_scf_sector_list import get_scf_sector_list

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [30 / 2.54, 20 / 2.54]


### Pre-define data paths, global variables etc.

In [3]:
# Get relevant data_paths for DLC and simulation result files
data_path              = fr'{os.getcwd()}\data'
geometry_file          = data_path +  r'\DA_P53_CD.xlsx'
point_angles  = [float(i) for i in range(0,359,15)] 

# Extract file for all DEM sums per geo, per elevation
DEM_data_path = fr'{os.getcwd()}\output\python_combined_DEM.xlsx'
df_xlsx = pd.read_excel(DEM_data_path)

# Define DEM variables
T_lifetime = 25.0
N_equivalent = 1e7
wohler_exp = 5.0
curve = SN_Curve_qats('D')
DFF = 3.0

### Extract hotspot geometry

In [4]:
geo_test_df = pd.read_excel(fr'{os.getcwd()}\unused\DA_P53_CD copy with extra geo.xlsx')
top_boat_landing_geo = create_geo_matrix(geo_test_df, point_angles, curve)[0]
# top_boat_landing_geo = geo_test_df.iloc[0]
elevation_boat_landing = top_boat_landing_geo['elevation']

### Define angles in both turbine frame and compass frame

In [5]:
# Translate the angles to compass angles to align with given SCF factor
global_angles = [float(key) for key in df_xlsx.iloc[0][1:].keys()] # 1: due to element 0 being 'mLat'
compass_angles = global_2_compass(global_angles)
adjusted_compass_angles, scf_sectors = get_scf_sector_list(top_boat_landing_geo, compass_angles)
adjusted_global_angles = compass_2_global(adjusted_compass_angles)
validate_df = pd.DataFrame(data=np.hstack((np.array([adjusted_compass_angles]).reshape(-1,1), 
                                np.array([scf_sectors]).reshape(-1,1), 
                                np.array([adjusted_global_angles]).reshape(-1,1))), 
                                columns = ['Adjusted compass angles', 'SCF','Adjusted global angles'])

### Extract pre computed DEM data - use boat landing geometry - find member_elevations of nodes above and below boat landing

In [6]:
''' The goal here is for this script to be independent of the geometries that were used during the initial DEM-sum calculations '''
member_elevations = np.array([float(key) for key in (df_xlsx['mLat'].values)])
dist = member_elevations - elevation_boat_landing

assert (np.any(dist >= 0.)), 'Geo in question is above the highest node - invalid for interpolation'
assert (np.any(dist < 0.)), 'Geo in question is below the lowest node - invalid for interpolation'

In [7]:
idx_above = np.ma.MaskedArray(dist, dist < 0.0).argmin() # mask out values below 0, and find the positive value closest to zero
idx_below = np.ma.MaskedArray(dist, dist >= 0.0).argmax() # mask out values above 0, and find the negative value closest to zero
idx_closest = np.abs(dist).argmin()

elevation_above = member_elevations[idx_above]
elevation_below = member_elevations[idx_below]
elevation_closest = member_elevations[idx_closest]
                             
print(f'The nodes from the DEM calculation closest to boat landing at {elevation_boat_landing:.2f} mLat is \n{elevation_above:.2f} @ mLat above \n{elevation_below:.2f} @ mLat below')
closest_is_above = elevation_closest > elevation_boat_landing
print(f'The closest of elevations is {elevation_closest}, {"above" if closest_is_above else "below"} the boat landing')

The nodes from the DEM calculation closest to boat landing at 9.69 mLat is 
11.20 @ mLat above 
6.50 @ mLat below
The closest of elevations is 11.2, above the boat landing


### Extract DEM from the neighbouring elevations

In [8]:
# Manually find DEM interpolated for 9.69 mLat by using manually chosen elevations
above_DEM = np.array(df_xlsx.iloc[idx_above][1:]) # remove the mLat dimension of the DataFrame - this will be a np.array with DEM for all (24) sectors
below_DEM = np.array(df_xlsx.iloc[idx_below][1:])

# Linear interpol; DEM_i = DEM_below + [dy/dx] * dx = DEM_below + [(delta DEM above/below) / (delta elevation above/below) ] * (elevation geo i - elevation below) or DEM_below + interpol_factor * delta DEM
interpol_factor = (elevation_boat_landing - elevation_below) / (elevation_above - elevation_below) 

## Use the interpolated DEM to calculate the factor to scale moment ranges with accordingly

In [38]:
# Calculate total DEM for above, current and below elevations, for all sectors (n_sectors, )-shaped
DEM_above_tot = ((T_lifetime / N_equivalent * above_DEM)**(1 / wohler_exp))
DEM_below_tot = ((T_lifetime / N_equivalent * below_DEM)**(1 / wohler_exp))
DEM_boat_landing_tot = DEM_below_tot + (DEM_above_tot - DEM_below_tot) * interpol_factor

### Find the reference DEM and scaling factor / scaled DEM in hotspot

In [39]:
# Find above and below
DEM_hotspot_vs_above = np.array(DEM_boat_landing_tot / DEM_above_tot) # per sector
f_hotspot_vs_above = DEM_hotspot_vs_above.max()
DEM_hotspot_vs_below = np.array(DEM_boat_landing_tot / DEM_below_tot)
f_hotspot_vs_below = DEM_hotspot_vs_below.max()

In [40]:
# Choose the reference according to closest elevation
# TODO should we choose worst, or the closest to 
# And how do we select the sector on the hotspot elevation BEFORE we know the worst DEM on that elevation? Just choose an angle with the highest SCF?

DEM_reference_factor = f_hotspot_vs_above if closest_is_above else f_hotspot_vs_below
sector_idx_for_worst_reference_DEM = DEM_hotspot_vs_above.argmax() if closest_is_above else DEM_hotspot_vs_below.argmax()
DEM_reference = DEM_above_tot[sector_idx_for_worst_reference_DEM] if closest_is_above else DEM_below_tot[sector_idx_for_worst_reference_DEM]
DEM_hotspot_interpolated = DEM_reference * DEM_reference_factor

print(f'DEM relationships above and below')
print(f'From report: above vs. hotspot 98.2 MNm * 1.00772 ~ {98.2 * 1.00772:.1f} MNm')
# print(f'Above : {f_hotspot_vs_above} at ang {global_angles[DEM_hotspot_vs_above.argmax() ]} / {compass_angles[DEM_hotspot_vs_above.argmax()]} degN')
# print(f'Below: {f_hotspot_vs_below} at ang {global_angles[ DEM_hotspot_vs_below.argmax() ] } / {compass_angles[DEM_hotspot_vs_below.argmax()]} degN')
print('#'*40)
print(f'Chosing factor {"above" if closest_is_above else "below"} as reference, with reference M_eq = {DEM_reference / 10**6:.2f} MNm at {compass_angles[DEM_hotspot_vs_above.argmax()]} degN')
print(f'Interpolated DEM at hotspot with factor {DEM_reference_factor:5f} gives M_scf = {DEM_hotspot_interpolated / 10**6:.2f} MNm')

DEM relationships above and below
From report: above vs. hotspot 98.2 MNm * 1.00772 ~ 99.0 MNm
########################################
Chosing factor above as reference, with reference M_eq = 96.46 MNm at 180.0 degN
Interpolated DEM at hotspot with factor 1.012851 gives M_scf = 97.70 MNm


## Load the markov matrices with moment ranges and counts

In [12]:
# Load the markov matrices
from utils.fastnumpyio import load as fastio_load 
markov_path = fr'{os.getcwd()}\output\total_markov_member' + '{}.npy'
mbr_to_el_map = {54: 22.4, 72: 11.2, 84: 6.5, 91: 4.275} # TODO this should be handled by the geometry file
markov_matrices = {}
for mbr in mbr_to_el_map.keys():
    path = markov_path.format(mbr)
    markov_matrices[mbr_to_el_map[mbr]] = np.array(fastio_load(path))

### Pick the markov matrix from the reference node, AT THE WORST SECTOR

In [41]:
reference_elevation = elevation_above if closest_is_above else elevation_below
markov_at_reference_elevation = markov_matrices[reference_elevation]
markov_reference = markov_at_reference_elevation[sector_idx_for_worst_reference_DEM]
markov_reference = markov_reference[ markov_reference[:, 0].argsort()]

### Check if it is possible to recreate the DEM at the reference elevation with the loaded ranges

In [72]:
reference_moment_ranges = markov_reference[:,[0]]
reference_counts = markov_reference[:,[1]]
DEM_reference_recreated = float((T_lifetime / N_equivalent * ((reference_moment_ranges.T)**wohler_exp).dot(reference_counts))**(1 / wohler_exp))

print(f'Recreated stats from the reference markov matrix at {elevation_closest:.2f} mLat:')
print(f'Total counts over {T_lifetime:.0f} years: {reference_counts.sum() * T_lifetime / 10**6:.2f} * 10^6')
print(f'Total bins with nonzero counts: {np.count_nonzero(reference_counts)}')
print(f'Smallest M_range: {reference_moment_ranges.min() * 1e-6:.2f} MNm') 
print(f'Median M_range: {np.median(reference_moment_ranges) * 1e-6:.2f} MNm') 
print(f'Largest M_range: {reference_moment_ranges.max() * 1e-6:.2f} MNm') 
print(f'Recreated DEM at reference for m = 5: {(DEM_reference_recreated) / 10**6 :.2f} MNm')
print(f'Original DEM at reference for m = 5: {DEM_reference / 10**6 :.2f} MNm')
print(f'It is {"possible" if ((DEM_reference - DEM_reference_recreated) < 1e-6) else "not possible"} to recreate DEM')

Recreated stats from the reference markov matrix at 11.20 mLat:
Total counts over 25 years: 163.97 * 10^6
Total bins with nonzero counts: 745768
Smallest M_range: 0.14 MNm
Median M_range: 46.18 MNm
Largest M_range: 401.38 MNm
Recreated DEM at reference for m = 5: 96.46 MNm
Original DEM at reference for m = 5: 96.46 MNm
It is possible to recreate DEM


### Rescale the moment ranges for the nearest hotspot according to the DEM relation, and find corresponding stress ranges

In [73]:
# Scale all moment ranges per sector according to relationships calculated above, or just one single factor determined by DEM max. Test below, above and closest
markov_hotspot = markov_reference.copy()
markov_hotspot[:, [0]] = markov_reference[:, [0]] * DEM_reference_factor # could use *= but this stays in control "compunding interest" when using notebooks
DEM_hotspot_recreated = ( T_lifetime / N_equivalent * ((markov_hotspot[:,[0]].T)**wohler_exp).dot(markov_hotspot[:,[1]]) )**(1 / wohler_exp)

print(f'Largest M_range: {markov_hotspot[:, 0].max() * 1e-6:.2f} MNm') 
print(f'Recreated DEM at hotspot for m = 5: {float(DEM_hotspot_recreated) / 10**6 :.2f} MNm')
print(f'Original DEM at hotspot for m = 5: {DEM_hotspot_interpolated / 10**6 :.2f} MNm')
print(f'It is {"possible" if ((DEM_hotspot_recreated - DEM_hotspot_interpolated) < 1e-6) else "not possible"} to recreate DEM')

Largest M_range: 406.53 MNm
Recreated DEM at hotspot for m = 5: 97.70 MNm
Original DEM at hotspot for m = 5: 97.70 MNm
It is possible to recreate DEM


## Use the hotspot moment ranges to calculate all stress ranges

In [78]:
stress_ranges_unscaled = markov_hotspot[:, [0]] * top_boat_landing_geo['D'] / (2 * top_boat_landing_geo['I'])

### Scale stress ranges according to SCF and effective thickness

In [88]:
t_eff = top_boat_landing_geo['t_eff']
alpha = (t_eff / 25.0)**curve.k

# Scale stress ranges according to SCF
scf_factor = np.max(scf_sectors) # TODO solve this for the closest point at hotspot elevation
print(t_eff)
print(alpha)
print(scf_factor)

stress_ranges_scaled = stress_ranges_unscaled * scf_factor * top_boat_landing_geo['alpha']

53.6
1.1647828195380305
1.502


In [103]:
# Use stress ranges in miner summation for damage (see reports)
stress_ranges_MPa = stress_ranges_scaled / 10**6
total_hotspot_counts = markov_hotspot[:,[1]] * T_lifetime
stress_ranges_and_counts = np.hstack( (stress_ranges_MPa, total_hotspot_counts ))
in_place_hotspot_utilization = curve.miner_sum(stress_ranges_and_counts) * DFF
print(f'Total in-place damage of hotspot = {in_place_hotspot_utilization * 100.0:.2f} %')

Total in-place damage of hotspot= 74.11 %


### Extra: recreating damage from the WTP report

In [109]:
# Note that this is not working for our hotspots of interest

# TODO this does not make sense - example values from report
uts = 420.0
print(f'0{0.59}: {curve.miner_sum(np.array([[00.59 * (uts / (uts - (83.96 + 84.55)/2)) * alpha, 1.1]])) * DFF:.2e}')
print(f'0{5.31}: {curve.miner_sum(np.array([[05.31 * (uts / (uts - (-7.51 + -2.20)/2)) * alpha, 301506.5]])) * DFF:.2e}')
print(f'{44.28}: {curve.miner_sum(np.array([[44.28 * (uts / (uts - (29.85 + 74.13)/2)) * alpha, 35.2]])) * DFF:.2e}')
print(f'{44.28}: {curve.miner_sum(np.array([[44.28 * (uts / (uts - (-100.73 + -56.45)/2)) * alpha, 89.3]])) * DFF:.2e}')
print(f'{150.54}: {curve.miner_sum(np.array([[150.54 * (uts / (uts - (-89.34 + 61.20)/2)) * alpha, 0.2]])) * DFF:.2e}')


00.59: 3.83e-16
05.31: 1.91e-06
44.28: 1.48e-05
44.28: 1.03e-05
150.54: 2.01e-06


In [105]:
tableE80 = pd.read_excel(fr'{os.getcwd()}\unused\tableE80.xlsx')

In [108]:
tableE80
for idx in range(tableE80.shape[0]):
    row = tableE80.iloc[idx]
    mean = (row["Smin,HS"] + row["Smax,HS"]) / 2
    print(f'{row["Srange,HS"]} tab: {row["Dair"]:.2e}')
    print(f'{row["Srange,HS"]} me raw: {curve.miner_sum( np.array([row["Srange,HS"], row["Srange,HS"] ])) * DFF:.2e}')
    
    print(f'{row["Srange,HS"]} me w mean scale no alpha: {curve.miner_sum( np.array([row["Srange,HS"] * (uts / (uts - mean)), row["Srange,HS"] ])) * DFF:.2e}')
    print(f'{row["Srange,HS"]} me w mean scale w alpha: {curve.miner_sum( np.array([row["Srange,HS"] * (uts / (uts - mean)) * alpha, row["Srange,HS"] ])) * DFF:.2e}')
    
    print('')




Bin          1.000000e+00
Mrange       1.590000e+00
Mmean       -2.625600e+02
Mmin         5.588000e+01
Mmax         5.627000e+01
Mrange.1     3.900000e-01
Smin,HS      8.396000e+01
Smax,HS      8.455000e+01
Srange,HS    5.900000e-01
Ntotal       1.100000e+00
Nair         1.100000e+00
Dair         1.270000e-16
Name: 0, dtype: float64
Bin          7.500000e+01
Mrange       1.590000e+00
Mmean        4.342000e+01
Mmin        -1.979000e+01
Mmax        -1.939000e+01
Mrange.1     3.900000e-01
Smin,HS     -2.973000e+01
Smax,HS     -2.914000e+01
Srange,HS    5.900000e-01
Ntotal       4.323206e+06
Nair         4.323206e+06
Dair         4.980000e-10
Name: 1, dtype: float64
Bin          1.490000e+02
Mrange       4.770000e+00
Mmean       -1.550600e+02
Mmin         2.890000e+01
Mmax         3.008000e+01
Mrange.1     1.180000e+00
Smin,HS      4.342000e+01
Smax,HS      4.519000e+01
Srange,HS    1.770000e+00
Ntotal       2.250999e+05
Nair         2.250999e+05
Dair         6.300000e-09
Name: 2, dtype: 